---
title: "Relations"
categories: [sqlalchemy]
image: "image.jpg"
---

How do define relationships without explicitly setting foreign keys.

In [1]:
import sqlalchemy as db

from sqlalchemy import create_engine, text
from sqlalchemy.orm import declarative_base, relationship, Session
Base = declarative_base()

In [2]:
class User(Base):
    __tablename__ = "user"
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String)
    products = relationship("Product", back_populates="user")

class Product(Base):
    __tablename__ = "product"
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String)
    user_id = db.Column(db.Integer, db.ForeignKey("user.id"), nullable=False)
    user = relationship("User", back_populates="products")

In [3]:
db_path = "sqlite:///test.db"
engine = db.create_engine(db_path)
Base.metadata.create_all(engine, checkfirst=True)

session = Session(bind=engine)
# enforce foreign key constraint in SQLite (off by default!)
session.execute(text("PRAGMA foreign_keys=on"));

In [4]:
user = User(name="Alice")
product1 = Product(name="A", user=user)
product2 = Product(name="B", user=user)

session.add(user)
session.commit()

Foreign key will be correctly assigned by sqlalchemy:

In [5]:
product1.user_id

1

Works also the other way round:

In [6]:
user = User(name="Kyle")
product3 = Product(name="A")
product4 = Product(name="B")

user.products = [product3, product4]

session.add(user)
session.commit()

Foreign key will be correctly assigned by sqlalchemy:

In [7]:
product3.user_id

2

### This does not work
The associated user.id will not be set correctly. It will be set to `None` which is the current value of `user.id` prior to committing.

In [8]:
user = User(name="Dave")

product5 = Product(name="C", user_id=user.id) # user.id is None at this point!
product6 = Product(name="D", user_id=user.id)

session.add_all([user, product5, product6])
session.commit()

IntegrityError: (sqlite3.IntegrityError) NOT NULL constraint failed: product.user_id
[SQL: INSERT INTO product (name, user_id) VALUES (?, ?) RETURNING id]
[parameters: ('C', None)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)